In [14]:
"""
Graph API

Limitations
You can only read a maximum of 100 feed posts with the limit field. 
If you try to read more than that you will get an error message to not exceed 100.

When you use /page-id/tagged to show the posts that tag this page, the results include 
posts from other pages only if those pages are authentic.

The available user agents allowed for these Graph API calls are subject to change without notice. 
If you are experiencing issues, you may want to change to a newer version of your particular user agent.
"""

'\nGraph API\n\nLimitations\nYou can only read a maximum of 100 feed posts with the limit field. \nIf you try to read more than that you will get an error message to not exceed 100.\n\nWhen you use /page-id/tagged to show the posts that tag this page, the results include \nposts from other pages only if those pages are authentic.\n\nThe available user agents allowed for these Graph API calls are subject to change without notice. \nIf you are experiencing issues, you may want to change to a newer version of your particular user agent.\n'

In [15]:
"""
Step 0- Harnessing Social Data - Connecting, Capturing, and Cleaning
"""

import requests 
import json
from pymongo import MongoClient

params = {'access_token': 'EAAE0OPeUvUcBAHr0mJC7rvdzlZAJzP27cZAuSTy2ZAN3wjAeCZAnKjiqLpxBgM8M9o1vUpCkTOzsD4lb4kVZAkuoKynQdVONFVYzZCeKvZBEVIIpfLgUhJRY6TpijFZBSD3mwnTqgZA57JhaUsQZBo8LnkS1aPuC7h8RcZD'}

client = MongoClient('localhost:27017')
db = client.facebook
collection_posts = db.posts
collection_comments = db.comments

In [ ]:
"""
Step 1 – data extraction
We will define Facebook endpoints, wich will be used to retrieve the data. 

In Graph API all the fields have to be explicity specified:
    ·id: the Id of the post
    ·message: the content of the post
    ·reactions: a list of reactions of this post
    ·shares: the number of shares
    ·from: the author name of the post
    ·caption: the caption of the post
    ·created_time: a timestap
    
In the comments_url we defined two fields:
    ·filter=stream: to get paginated results
    ·limit: the limit of the maximum allowed
"""

# first endpoint will be used to extract all the posts:
page_url = 'https://graph.facebook.com/v2.8/Google/feed?fields=id,message,reactions,shares,from,caption,created_time,likes.summary(true)'

# second endpoint to extract all comments
comments_url = 'https://graph.facebook.com/v2.8/{post_id}/comments?filter=stream&limit=100'

# Get the first page of posts
result = requests.get(page_url, params = params)

posts = result.json()

# Until the response is not empty (there are more posts in response) try to get all the posts and related comments:
while True:
    try:
    ###Retrieve one post
        for element in posts['data']:
            collection_posts.insert(element)
            ####Retrieve all comments for this post
            this_comment_url = comments_url.replace("{post_id}",element['id'])
            comments = requests.get(this_comment_url, params = params).json()
            
            #loop through all comments until the response is empty (there are no more comments)
            while ('paging' in comments and 'cursors' in comments['paging'] and 'after' in comments['paging']['cursors']):
                ###Iterate through all comments
                for comment in comments['data']:
                    comment['post_id'] = element['id']
                    collection_comments.insert(comment)
                
                comments = requests.get(this_comment_url + '&after=' + comments['paging']['cursors']['after'], params = params).json()
 
        ####Go to the next page in feed
        posts = requests.get(posts['paging']['next']).json()
    except (KeyError, e):
        print(e)
        break

In [23]:
import pandas as pd
import nltk
posts_data = [] 
comments_data = [] 

for doc in collection_posts.find({}): 
    try: 
        posts_data.append((doc['message'],doc['created_time'],doc['likes']['summary']['total_count'],doc['shares']['count'],doc['id'])) 
    except: 
#        print("No message") 
        pass
    
for comment in collection_comments.find({}): 
    try: 
        comments_data.append((comment['message'],comment['created_time'],comment['post_id'])) 
    except: 
        pass
    
#create dataframes
df_posts = pd.DataFrame(posts_data) 
df_posts.columns = ['message','created_time','likes','shares','post_id'] 
df_comments = pd.DataFrame(comments_data) 
df_comments.columns = ['message','created_time','post_id']

In [45]:
print("row: ", df_posts.shape[0], " col: ", df_posts.shape[1])
print("row: ", df_comments.shape[0], " col: ", df_comments.shape[1])

row:  2425  col:  5
row:  659119  col:  3
